모델 에러에 대한 대처를 어떻게 할 것인가

# 최종

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install transformers[torch] accelerate -U
!pip install datasets
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 31.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

# 데이터 불러오기

In [5]:
DF = pd.read_csv('/content/drive/MyDrive/testData.csv')
DF['product'] = DF['product'].replace('Money transfer, virtual currency, or money service', 'Money transfer')
DF = DF[['product','issue','consumer complaint narrative']]
DF

,product,issue,consumer complaint narrative
0,Credit reporting,Incorrect information on your report,I'm filing this complaint to the credit bureau...
1,Credit card,Trouble using your card,"XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX, CA XX..."
2,Credit reporting,Incorrect information on your report,I find it unacceptable that there are public r...
3,Debt collection,Written notification about debt,"XXXX - This is where I beg the difference, you..."
4,Credit reporting,Problem with a company's investigation into an...,I found inaccurate and incorrect data on my cr...
...,...,...,...
4995,Credit reporting,Incorrect information on your report,This is my fifth endeavor to tell you that I a...
4996,Credit reporting,Improper use of your report,In accordance with the Fair Credit Reporting A...
4997,Vehicle loan or lease,Managing the loan or lease,MY lease ender I bought out lease and finaced ...
4998,Credit reporting,Incorrect information on your report,According to the Pennsylvania Department of Tr...


In [6]:
train_df, test_df = train_test_split(DF, test_size=1000, stratify=DF['product'])
DF = test_df.reset_index()

# 자연어처리

In [7]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
data = list(DF['consumer complaint narrative'])
clean = []
for narrative in (data):
 clean_narrative = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…\"\“》XXXX{}]', '', narrative)
 clean.append(clean_narrative)
DF['narrative'] = clean

#소문자변환
lower = [text.lower() for text in DF['narrative'].values]
DF['narrative'] = lower

#불용어 제거
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

DF["narrative"]=DF["narrative"].apply(lambda x: remove_stopwords(x))

DF

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


,index,product,issue,consumer complaint narrative,narrative
0,3217,Credit reporting,Improper use of your report,How are you? I hope all is well. I am writing ...,hope well writing recently reviewed latest cre...
1,2506,Credit reporting,Improper use of your report,In Accordance with the Fair Credit Reporting A...,accordance fair credit reporting act bank acco...
2,196,Credit reporting,Problem with a company's investigation into an...,"Dear Legal Department, This credit dispute is ...",dear legal department credit dispute submitted...
3,2625,Debt collection,Written notification about debt,RE : Cease and Desist MIDLAND CREDIT MANAG cur...,cease desist midland credit manag currently ac...
4,3852,Credit reporting,Problem with a company's investigation into an...,"After currently reviewing my consumer report, ...",currently reviewing consumer report come atten...
...,...,...,...,...,...
995,466,Credit reporting,Improper use of your report,I did not authorize anyone employed by these c...,authorize anyone employed companies make inqui...
996,2543,Mortgage,Trouble during payment process,"Since XX/XX/XXXX, I have been making Principal...",since making principal payments addition regul...
997,2542,Debt collection,Attempts to collect debt not owed,The original credit card company charged off m...,original credit card company charged account b...
998,4451,Credit reporting,Incorrect information on your report,It has been brought to my attention that you a...,brought attention blocking files theyre sent f...


# 제품분류 전처리

## 1. 머신러닝 전처리

In [8]:
from sklearn.preprocessing import LabelEncoder
DF['product_L'] = LabelEncoder().fit_transform(DF['product'])

In [ ]:
DF['product_L'].value_counts()

product_L
2    641
3    126
1     72
0     59
5     50
7     17
4     16
8     13
6      6
Name: count, dtype: int64

In [ ]:
DF['product'].value_counts()

product
Credit reporting                                      3203
Debt collection                                        631
Credit card                                            359
Checking or savings account                            293
Mortgage                                               251
Student loan                                            86
Money transfer, virtual currency, or money service      80
Vehicle loan or lease                                   66
Payday loan                                             31
Name: count, dtype: int64

In [9]:
import joblib
loaded_model = joblib.load('/content/drive/MyDrive/ML_model')
loaded_vectorizer = joblib.load('/content/drive/MyDrive/vectorizer.joblib')

X = DF['narrative']
from sklearn.feature_extraction.text import CountVectorizer
X_Count = loaded_vectorizer.transform(X)
X_Count = X_Count.astype(np.float64)
X_Count.dtype

dtype('float64')

## 3. 딥러닝 전처리

In [ ]:
A = pd.get_dummies(DF['product']).astype('int')
DF = pd.concat([DF, A], axis = 1)
DF.head()

,product,issue,consumer complaint narrative,narrative,Checking or savings account,Credit card,Credit reporting,Debt collection,Money transfer,Mortgage,Payday loan,Student loan,Vehicle loan or lease
0,Credit reporting,Incorrect information on your report,I'm filing this complaint to the credit bureau...,im filing complaint credit bureau order notify...,0,0,1,0,0,0,0,0,0
1,Credit card,Trouble using your card,"XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX, CA XX...",ca 2018 barclays bank delaware dear barclays b...,0,1,0,0,0,0,0,0,0
2,Credit reporting,Incorrect information on your report,I find it unacceptable that there are public r...,find unacceptable public records credit report...,0,0,1,0,0,0,0,0,0
3,Debt collection,Written notification about debt,"XXXX - This is where I beg the difference, you...",beg difference certainly wrong rules fdcpa deb...,0,0,0,1,0,0,0,0,0
4,Credit reporting,Problem with a company's investigation into an...,I found inaccurate and incorrect data on my cr...,found inaccurate incorrect data credit report ...,0,0,1,0,0,0,0,0,0


In [ ]:
import joblib
loaded_tokenizer = joblib.load('/content/drive/MyDrive/tokenizer.joblib')
from tensorflow.keras.preprocessing.text import Tokenizer
loaded_tokenizer.fit_on_texts(DF['narrative'])
sequences = loaded_tokenizer.texts_to_sequences(DF['narrative'])
len(sequences)

5000

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sequences = pad_sequences(sequences,
                        maxlen = 150,
                        padding = 'post',
                        truncating = 'post')

In [ ]:
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
DF

,product,issue,consumer complaint narrative,narrative,product_L,Predicted Label,Predicted product,Checking or savings account,Credit card,Credit reporting,Debt collection,"Money transfer, virtual currency, or money service",Mortgage,Payday loan,Student loan,Vehicle loan or lease
0,Credit reporting,Incorrect information on your report,I'm filing this complaint to the credit bureau...,im filing complaint credit bureau order notify...,2,2,Credit reporting,0,0,1,0,0,0,0,0,0
1,Credit card,Trouble using your card,"XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX, CA XX...",ca 2018 barclays bank delaware dear barclays b...,1,1,Credit card,0,1,0,0,0,0,0,0,0
2,Credit reporting,Incorrect information on your report,I find it unacceptable that there are public r...,find unacceptable public records credit report...,2,2,Credit reporting,0,0,1,0,0,0,0,0,0
3,Debt collection,Written notification about debt,"XXXX - This is where I beg the difference, you...",beg difference certainly wrong rules fdcpa deb...,3,3,Debt collection,0,0,0,1,0,0,0,0,0
4,Credit reporting,Problem with a company's investigation into an...,I found inaccurate and incorrect data on my cr...,found inaccurate incorrect data credit report ...,2,2,Credit reporting,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Credit reporting,Incorrect information on your report,This is my fifth endeavor to tell you that I a...,fifth endeavor tell victim identity theft comp...,2,2,Credit reporting,0,0,1,0,0,0,0,0,0
4996,Credit reporting,Improper use of your report,In accordance with the Fair Credit Reporting A...,accordance fair credit reporting act account e...,2,2,Credit reporting,0,0,1,0,0,0,0,0,0
4997,Vehicle loan or lease,Managing the loan or lease,MY lease ender I bought out lease and finaced ...,lease ender bought lease finaced thru ally 202...,8,8,Vehicle loan or lease,0,0,0,0,0,0,0,0,1
4998,Credit reporting,Incorrect information on your report,According to the Pennsylvania Department of Tr...,according pennsylvania department transportati...,2,8,Vehicle loan or lease,0,0,1,0,0,0,0,0,0


In [ ]:
y =DF[['Checking or savings account', 'Credit card', 'Credit reporting',
       'Debt collection', 'Money transfer', 'Mortgage', 'Payday loan',
       'Student loan', 'Vehicle loan or lease']].values
y = y.astype(np.float32)
y

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

# 제품 분류

## 1.머신러닝

In [10]:
loaded_model.score(X_Count, DF['product_L'])

0.871

## 2.딥러닝

In [ ]:
model = load_model('/content/drive/MyDrive/best_model_Product')
print("\n 테스트 정확도: %.4f" % (model.evaluate(sequences, y)[1]))
#머신러닝 모델을 사용

157/157 [==============================] - 3s 16ms/step - loss: 0.4815 - accuracy: 0.8366

 테스트 정확도: 0.8366


# 컬럼추가

In [11]:
y_pred =loaded_model.predict(X_Count)
A = pd.DataFrame(y_pred, columns=['Predicted Label'])

In [12]:
DF = pd.concat([DF, A],axis = 1)
DF

,index,product,issue,consumer complaint narrative,narrative,product_L,Predicted Label
0,3217,Credit reporting,Improper use of your report,How are you? I hope all is well. I am writing ...,hope well writing recently reviewed latest cre...,2,2
1,2506,Credit reporting,Improper use of your report,In Accordance with the Fair Credit Reporting A...,accordance fair credit reporting act bank acco...,2,2
2,196,Credit reporting,Problem with a company's investigation into an...,"Dear Legal Department, This credit dispute is ...",dear legal department credit dispute submitted...,2,2
3,2625,Debt collection,Written notification about debt,RE : Cease and Desist MIDLAND CREDIT MANAG cur...,cease desist midland credit manag currently ac...,3,3
4,3852,Credit reporting,Problem with a company's investigation into an...,"After currently reviewing my consumer report, ...",currently reviewing consumer report come atten...,2,2
...,...,...,...,...,...,...,...
995,466,Credit reporting,Improper use of your report,I did not authorize anyone employed by these c...,authorize anyone employed companies make inqui...,2,2
996,2543,Mortgage,Trouble during payment process,"Since XX/XX/XXXX, I have been making Principal...",since making principal payments addition regul...,5,5
997,2542,Debt collection,Attempts to collect debt not owed,The original credit card company charged off m...,original credit card company charged account b...,3,3
998,4451,Credit reporting,Incorrect information on your report,It has been brought to my attention that you a...,brought attention blocking files theyre sent f...,2,2


In [12]:
y_test = DF['product_L']
y_pred = DF['Predicted Label']
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
print(confusion_matrix(y_test, y_pred))
print('accuracy  : %.4f' % accuracy_score(y_test, y_pred))
print('-' * 20)
# Precision : 예측이 test 인 것 중 실제도 test
# average=None, 각 class 별로 precision을 구해라
print(precision_score(y_test, y_pred, average=None))
print('precision (micro): %.4f' % precision_score(y_test, y_pred, average='micro'))
print('precision (macro): %.4f' % precision_score(y_test, y_pred, average='macro'))
print('precision (weighted): %.4f' % precision_score(y_test, y_pred, average='weighted'))

[[ 51   5   0   2   1   0   0   0   0]
 [  3  60   8   1   0   0   0   0   0]
 [  1  12 586  36   0   3   0   0   3]
 [  1   1  33  88   0   3   0   0   0]
 [  2   2   0   0  12   0   0   0   0]
 [  1   1   1   0   0  46   1   0   0]
 [  0   2   1   1   0   0   2   0   0]
 [  0   1   2   2   0   1   0  11   0]
 [  0   0   1   1   1   1   0   0   9]]
accuracy  : 0.8650
--------------------
[0.86440678 0.71428571 0.92721519 0.67175573 0.85714286 0.85185185
 0.66666667 1.         0.75      ]
precision (micro): 0.8650
precision (macro): 0.8115
precision (weighted): 0.8685


In [13]:
label_mapping = {
    0: 'Checking or savings account',
    1: 'Credit card',
    2: 'Credit reporting',
    3: 'Debt collection',
    4: 'Money transfer',
    5: 'Mortgage',
    6: 'Payday loan',
    7: 'Student loan',
    8: 'Vehicle loan or lease'
}

DF['Predicted product'] = DF['Predicted Label'].map(label_mapping)

In [14]:
DF

,index,product,issue,consumer complaint narrative,narrative,product_L,Predicted Label,Predicted product
0,3848,Credit card,Problem with a purchase shown on your statement,On XX/XX/2023 I checked into the XXXX XXXX for...,2023 checked nights sponsor texas checking cos...,1,1,Credit card
1,4296,Debt collection,Communication tactics,Company contacted me last week regarding a leg...,company contacted last week regarding legal ma...,3,3,Debt collection
2,1,Credit card,Trouble using your card,"XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX, CA XX...",ca 2018 barclays bank delaware dear barclays b...,1,1,Credit card
3,2699,Debt collection,Written notification about debt,I have been disputing this account for some ti...,disputing account time credit management sent ...,3,3,Debt collection
4,992,Credit card,Getting a credit card,On XX/XX/18 a fraudulent credit card was obtai...,18 fraudulent credit card obtained td bank na ...,1,1,Credit card
...,...,...,...,...,...,...,...,...
995,4049,Credit reporting,Incorrect information on your report,"Bank XXXX XXXX, They have not done their due d...",bank done due diligence refuse go records reco...,2,2,Credit reporting
996,397,Credit reporting,Unable to get your credit report or credit score,I just spent {$29.00} to obtain an updated cre...,spent 2900 obtain updated credit score transun...,2,2,Credit reporting
997,4514,Credit reporting,Improper use of your report,In accordance with the Fair Credit Reporting a...,accordance fair credit reporting act list acco...,2,2,Credit reporting
998,834,Credit reporting,Problem with a company's investigation into an...,Ive been trying to dispute several accounts on...,ive trying dispute several accounts report dis...,2,2,Credit reporting


In [15]:
DF['product'].value_counts()

product
Credit reporting               641
Debt collection                126
Credit card                     72
Checking or savings account     59
Mortgage                        50
Student loan                    17
Money transfer                  16
Vehicle loan or lease           13
Payday loan                      6
Name: count, dtype: int64

In [16]:
DF['Predicted product'].value_counts()

Predicted product
Credit reporting               632
Debt collection                131
Credit card                     84
Checking or savings account     59
Mortgage                        54
Money transfer                  14
Vehicle loan or lease           12
Student loan                    11
Payday loan                      3
Name: count, dtype: int64

In [14]:
DF_Final = DF.copy()

# 모델 불러오기

In [15]:
import torch
from transformers import AutoModelForSequenceClassification
#Credit reporting 모델
tokenizer_Credit_Reporting = joblib.load('/content/drive/MyDrive/Credit_tokenizer.joblib')
model_Credit_Reporting = load_model('/content/drive/MyDrive/best_model_Credit_reporting')

#Credit card 모델
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_Credit_Card = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/best_model_Credit_card").to(device)

#Debt collection 모델
model_Debt_collection = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/best_model_Debt_collection").to(device)

#Checking or savings account 모델
model_Checking = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/best_model_Checking").to(device)

#Mortgage 모델
model_Mortgage = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/best_model_Mortgage").to(device)

#Student loan 모델
model_Student_loan = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/best_model_Student_loan").to(device)

#Money transfer 모델
model_Money_transfer = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/best_model_Money_transfer").to(device)

#Vehicle loan or lease 모델
model_Vehicle_loan = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/best_model_Vehicle_loan").to(device)

#Payday loan 모델
model_Payday_loan = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/best_model_Payday_loan").to(device)

# 트랜스포머 준비요소

In [16]:
DF = DF[['narrative','Predicted product']]
DF.columns = ['text','product']
DF

,text,product
0,hope well writing recently reviewed latest cre...,Credit reporting
1,accordance fair credit reporting act bank acco...,Credit reporting
2,dear legal department credit dispute submitted...,Credit reporting
3,cease desist midland credit manag currently ac...,Debt collection
4,currently reviewing consumer report come atten...,Credit reporting
...,...,...
995,authorize anyone employed companies make inqui...,Credit reporting
996,since making principal payments addition regul...,Mortgage
997,original credit card company charged account b...,Debt collection
998,brought attention blocking files theyre sent f...,Credit reporting


In [17]:
import transformers
transformers.logging.set_verbosity_error()
import warnings; warnings.filterwarnings('ignore')
import os; os.environ['WANDB_DISABLED'] = 'true'
from datasets import Dataset,Features,Value,ClassLabel, DatasetDict


#토큰화
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer
from transformers import AutoTokenizer, Trainer, TrainingArguments

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)


#평가함수 정의
import evaluate
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# TrainingArguments
eval_args = TrainingArguments(
    output_dir="result",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

# Trainer
a = 0
trainer_Credit_card = Trainer(
    model= model_Credit_Card,
    args=eval_args,
    eval_dataset=a,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer_Debt_collection = Trainer(
    model= model_Debt_collection,
    args=eval_args,
    eval_dataset=a,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer_Checking = Trainer(
    model= model_Checking,
    args=eval_args,
    eval_dataset=a,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer_Mortgage = Trainer(
    model= model_Mortgage,
    args=eval_args,
    eval_dataset=a,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer_Student_loan = Trainer(
    model= model_Student_loan,
    args=eval_args,
    eval_dataset=a,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer_Money_transfer = Trainer(
    model= model_Money_transfer,
    args=eval_args,
    eval_dataset=a,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer_Vehicle_loan = Trainer(
    model= model_Vehicle_loan,
    args=eval_args,
    eval_dataset=a,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer_Payday_loan = Trainer(
    model= model_Payday_loan,
    args=eval_args,
    eval_dataset=a,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

# 맵핑

In [18]:
#맵핑
#Credit_reporting
Credit_reporting_mapping = {
    0: 'Credit monitoring or identity theft protection services',
    1: 'Improper use of your report',
    2: 'Incorrect information on your report',
    3: "Problem with a company's investigation into an existing problem",
    4: 'Problem with fraud alerts or security freezes',
    5: 'Unable to get your credit report or credit score',
}

#Credit_card
Credit_card_mapping = {
    0: 'Advertising and marketing, including promotional offers',
    1: 'Closing your account',
    2: 'Fees or interest',
    3: "Getting a credit card",
    4: 'Other features, terms, or problems',
    5: 'Problem when making payments',
    6: "Problem with a company's investigation into an existing problem",
    7: 'Problem with a purchase shown on your statement',
    8: 'Trouble using your card',
}

#Debt_collection
Debt_collection_mapping = {
    0: 'Attempts to collect debt not owed',
    1: 'Communication tactics',
    2: "False statements or representation",
    3: 'Took or threatened to take negative or legal action',
    4: 'Written notification about debt',
}

#Checking
Checking_mapping = {
    0: 'Closing an account',
    1: 'Managing an account',
    2: 'Opening an account',
    3: "Problem caused by your funds being low",
    4: 'Problem with a lender or other company charging your account',
}

#Mortgage
Mortgage_mapping = {
    0: 'Applying for a mortgage or refinancing an existing mortgage',
    1: 'Closing on a mortgage',
    2: 'Struggling to pay mortgage',
    3: "Trouble during payment process",
}

#Student_loan
Student_loan_mapping = {
    0: 'Dealing with your lender or servicer',
    1: 'Incorrect information on your report',
    2: 'Struggling to repay your loan',
}

#Money_transfer
Money_transfer_mapping = {
    0: 'Fraud or scam',
    1: 'Managing, opening, or closing your mobile wallet account',
    2: 'Other transaction problem',
}

#Vehicle_loan
Vehicle_loan_mapping = {
    0: 'Managing the loan or lease',
    1: 'Problems at the end of the loan or lease',
    2: 'Struggling to pay your loan',
}

#Payday_loan
Payday_loan_mapping = {
    0: "Charged fees or interest you didn't expect",
    1: 'Struggling to pay your loan',
    2: 'Problem when making payments',
}

# 이슈분류

In [19]:
#리스트
Pre = []

#이슈분류
for i in range(len(DF)):
    row = DF.iloc[[i]]
    #Credit reporting
    if row['product'].values[0] == 'Credit reporting':
      tokenizer_Credit_Reporting.fit_on_texts(row['text'])
      sequences = tokenizer_Credit_Reporting.texts_to_sequences(row['text'])
      sequences = pad_sequences(sequences,
                        maxlen = 100,
                        padding = 'post',
                        truncating = 'post')

      y_pred = np.argmax(model_Credit_Reporting.predict(sequences), axis = 1)
      y_pred = Credit_reporting_mapping[y_pred[0]]
      Pre.append(y_pred)

    #Credit card
    elif row['product'].values[0] == 'Credit card':
      testdts = Dataset.from_pandas(row[['text']])
      corpus = DatasetDict({"validation" : testdts })
      tokenized_corpus = corpus.map(preprocess_function, batched=True)
      a = tokenized_corpus["validation"]
      pred_output = trainer_Credit_card.predict(tokenized_corpus["validation"])
      y_pred = np.argmax(pred_output.predictions,axis=1)
      y_pred = np.array([Credit_card_mapping[pred] for pred in y_pred])[0]
      Pre.append(y_pred)

    #'Debt collection
    elif row['product'].values[0] == 'Debt collection':
      testdts = Dataset.from_pandas(row[['text']])
      corpus = DatasetDict({"validation" : testdts })
      tokenized_corpus = corpus.map(preprocess_function, batched=True)
      pred_output = trainer_Debt_collection.predict(tokenized_corpus["validation"])
      y_pred = np.argmax(pred_output.predictions,axis=1)
      y_pred = np.array([Debt_collection_mapping[pred] for pred in y_pred])[0]
      Pre.append(y_pred)

    #Checking or savings account
    elif row['product'].values[0] == 'Checking or savings account':
      testdts = Dataset.from_pandas(row[['text']])
      corpus = DatasetDict({"validation" : testdts })
      tokenized_corpus = corpus.map(preprocess_function, batched=True)
      a = tokenized_corpus["validation"]
      pred_output = trainer_Checking.predict(tokenized_corpus["validation"])
      y_pred = np.argmax(pred_output.predictions,axis=1)
      y_pred = np.array([Checking_mapping[pred] for pred in y_pred])[0]
      Pre.append(y_pred)

    #Mortgage
    elif row['product'].values[0] == 'Mortgage':
      testdts = Dataset.from_pandas(row[['text']])
      corpus = DatasetDict({"validation" : testdts })
      tokenized_corpus = corpus.map(preprocess_function, batched=True)
      pred_output = trainer_Mortgage.predict(tokenized_corpus["validation"])
      y_pred = np.argmax(pred_output.predictions,axis=1)
      y_pred = np.array([Mortgage_mapping[pred] for pred in y_pred])[0]
      Pre.append(y_pred)

    #Student loan
    elif row['product'].values[0] == 'Student loan':
      testdts = Dataset.from_pandas(row[['text']])
      corpus = DatasetDict({"validation" : testdts })
      tokenized_corpus = corpus.map(preprocess_function, batched=True)
      pred_output = trainer_Student_loan.predict(tokenized_corpus["validation"])
      y_pred = np.argmax(pred_output.predictions,axis=1)
      y_pred = np.array([Student_loan_mapping[pred] for pred in y_pred])[0]
      Pre.append(y_pred)

    #Money transfer
    elif row['product'].values[0] == 'Money transfer':
      testdts = Dataset.from_pandas(row[['text']])
      corpus = DatasetDict({"validation" : testdts })
      tokenized_corpus = corpus.map(preprocess_function, batched=True)
      pred_output = trainer_Money_transfer.predict(tokenized_corpus["validation"])
      y_pred = np.argmax(pred_output.predictions,axis=1)
      y_pred = np.array([Money_transfer_mapping[pred] for pred in y_pred])[0]
      Pre.append(y_pred)

    #Vehicle loan or lease
    elif row['product'].values[0] == 'Vehicle loan or lease':
      testdts = Dataset.from_pandas(row[['text']])
      corpus = DatasetDict({"validation" : testdts })
      tokenized_corpus = corpus.map(preprocess_function, batched=True)
      pred_output = trainer_Vehicle_loan.predict(tokenized_corpus["validation"])
      y_pred = np.argmax(pred_output.predictions,axis=1)
      y_pred = np.array([Vehicle_loan_mapping[pred] for pred in y_pred])[0]
      Pre.append(y_pred)

    #Payday loan
    elif row['product'].values[0] == 'Payday loan':
      testdts = Dataset.from_pandas(row[['text']])
      corpus = DatasetDict({"validation" : testdts })
      tokenized_corpus = corpus.map(preprocess_function, batched=True)
      pred_output = trainer_Payday_loan.predict(tokenized_corpus["validation"])
      y_pred = np.argmax(pred_output.predictions,axis=1)
      y_pred = np.array([Payday_loan_mapping[pred] for pred in y_pred])[0]
      Pre.append(y_pred)

    else:
      continue

1/1 [==============================] - 0s 36ms/step


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

1/1 [==============================] - 0s 18ms/step


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

1/1 [==============================] - 0s 21ms/step


...

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

1/1 [==============================] - 0s 20ms/step


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

1/1 [==============================] - 0s 20ms/step


In [20]:
print(len(Pre))
DF_Final['Predicted issue'] = Pre

1000


In [23]:
DF_Final.head(2)

,index,product,issue,consumer complaint narrative,narrative,product_L,Predicted Label,Predicted product,Predicted issue
0,3217,Credit reporting,Improper use of your report,How are you? I hope all is well. I am writing ...,hope well writing recently reviewed latest cre...,2,2,Credit reporting,Problem with a company's investigation into an...
1,2506,Credit reporting,Improper use of your report,In Accordance with the Fair Credit Reporting A...,accordance fair credit reporting act bank acco...,2,2,Credit reporting,Improper use of your report


In [24]:
DF_Final = DF_Final[['narrative','product','Predicted product','issue','Predicted issue']]
DF_Final

,narrative,product,Predicted product,issue,Predicted issue
0,hope well writing recently reviewed latest cre...,Credit reporting,Credit reporting,Improper use of your report,Problem with a company's investigation into an...
1,accordance fair credit reporting act bank acco...,Credit reporting,Credit reporting,Improper use of your report,Improper use of your report
2,dear legal department credit dispute submitted...,Credit reporting,Credit reporting,Problem with a company's investigation into an...,Improper use of your report
3,cease desist midland credit manag currently ac...,Debt collection,Debt collection,Written notification about debt,Attempts to collect debt not owed
4,currently reviewing consumer report come atten...,Credit reporting,Credit reporting,Problem with a company's investigation into an...,Improper use of your report
...,...,...,...,...,...
995,authorize anyone employed companies make inqui...,Credit reporting,Credit reporting,Improper use of your report,Improper use of your report
996,since making principal payments addition regul...,Mortgage,Mortgage,Trouble during payment process,Trouble during payment process
997,original credit card company charged account b...,Debt collection,Debt collection,Attempts to collect debt not owed,Attempts to collect debt not owed
998,brought attention blocking files theyre sent f...,Credit reporting,Credit reporting,Incorrect information on your report,Problem with a company's investigation into an...


In [ ]:
DF_Final['product'].value_counts()

product
Credit reporting               1922
Debt collection                 378
Credit card                     215
Checking or savings account     176
Mortgage                        150
Student loan                     52
Money transfer                   48
Vehicle loan or lease            40
Payday loan                      19
Name: count, dtype: int64

In [ ]:
DFA = DF_Final.copy()

In [ ]:
DFA[DFA['product']=='Debt collection']

,narrative,product,issue,Predicted product,예측이슈
0,letter formal compliant reporting incomplete c...,Debt collection,Written notification about debt,Credit reporting,Problem with a company's investigation into an...
9,writing regarding account number associated na...,Debt collection,Attempts to collect debt not owed,Credit reporting,Problem with a company's investigation into an...
13,certified letter sent ad astra asking document...,Debt collection,Written notification about debt,Credit reporting,Incorrect information on your report
22,cease desist midland credit manag currently ac...,Debt collection,Written notification about debt,Debt collection,[Attempts to collect debt not owed]
32,repeatedly asked vance huffman validate collec...,Debt collection,Written notification about debt,Debt collection,[Written notification about debt]
...,...,...,...,...,...
973,writting report multiple violations rights con...,Debt collection,Attempts to collect debt not owed,Credit reporting,Problem with a company's investigation into an...
974,amsher collection services collecting charging...,Debt collection,False statements or representation,Debt collection,[False statements or representation]
976,prince parker associates contacted multiple ti...,Debt collection,Attempts to collect debt not owed,Debt collection,[Attempts to collect debt not owed]
988,made several attempts clear performed unauthor...,Debt collection,Attempts to collect debt not owed,Credit card,[Problem with a purchase shown on your statement]


In [26]:
y_test = DF_Final['issue']
y_pred = DF_Final['Predicted issue']
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
#print(confusion_matrix(y_test, y_pred))
print('accuracy  : %.4f' % accuracy_score(y_test, y_pred))
print('-' * 20)
# Precision : 예측이 test 인 것 중 실제도 test
# average=None, 각 class 별로 precision을 구해라
#print(precision_score(y_test, y_pred, average=None))
print('precision (micro): %.4f' % precision_score(y_test, y_pred, average='micro'))
print('precision (macro): %.4f' % precision_score(y_test, y_pred, average='macro'))
print('precision (weighted): %.4f' % precision_score(y_test, y_pred, average='weighted'))

accuracy  : 0.7040
--------------------
precision (micro): 0.7040
precision (macro): 0.6248
precision (weighted): 0.7092
